In [1]:
import logging
# Suppress PyPSA INFO messages (keep warnings and errors)
logging.getLogger('pypsa').setLevel(logging.WARNING)

import pypsa
import pandas as pd
import numpy as np
import gymnasium as gym
from gymnasium import spaces

import gc
import psutil
import matplotlib.pyplot as plt

import neptune

from torch.utils.data import TensorDataset, DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F

import random

import os
#import ipdb

/Users/antoniagrindrod/miniconda3/envs/pypsa-earth-rl/lib/python3.11/site-packages/swagger_spec_validator/validator12.py:18: DeprecationWarning: jsonschema.RefResolver is deprecated as of v4.18.0, in favor of the https://github.com/python-jsonschema/referencing library, which provides more compliant referencing behavior as well as more flexible APIs for customization. A future release will remove RefResolver. Please file a feature request (on referencing) if you are missing an API for the kind of customization you need.
  from jsonschema import RefResolver
/Users/antoniagrindrod/miniconda3/envs/pypsa-earth-rl/lib/python3.11/site-packages/swagger_spec_validator/validator12.py:18: DeprecationWarning: jsonschema.RefResolver is deprecated as of v4.18.0, in favor of the https://github.com/python-jsonschema/referencing library, which provides more compliant referencing behavior as well as more flexible APIs for customization. A future release will remove RefResolver. Please file a feature re

In [2]:
def fix_artificial_lines_reasonable(network):
    """
    Fix artificial lines with reasonable capacity values:
    - s_nom = based on connected bus demand (with safety factor)
    - s_nom_extendable = False (non-extendable)
    - Keep capacity high enough to meet demand
    """
    print("=== FIXING ARTIFICIAL LINES WITH REASONABLE CAPACITY ===")

    # Find artificial lines
    artificial_lines = [line for line in network.lines.index
                       if any(keyword in str(line).lower() for keyword in ['new', '<->', 'artificial'])]

    if not artificial_lines:
        # If no artificial lines found by name, look for lines with s_nom=0
        # which is often a sign of artificial lines
        zero_capacity_lines = network.lines[network.lines.s_nom == 0].index.tolist()
        if zero_capacity_lines:
            artificial_lines = zero_capacity_lines

    print(f"Found {len(artificial_lines)} artificial lines to fix:")

    # Get maximum demand per bus across all snapshots
    bus_max_demand = {}
    for bus in network.buses.index:
        bus_demand = 0
        for load_name, load in network.loads.iterrows():
            if load.bus == bus and load_name in network.loads_t.p_set.columns:
                bus_demand = max(bus_demand, network.loads_t.p_set[load_name].max())
        bus_max_demand[bus] = bus_demand

    # Fix each artificial line with reasonable capacity
    for line_name in artificial_lines:
        # Get connected buses
        bus0 = network.lines.loc[line_name, 'bus0']
        bus1 = network.lines.loc[line_name, 'bus1']

        # Get maximum demand at these buses
        bus0_demand = bus_max_demand.get(bus0, 0)
        bus1_demand = bus_max_demand.get(bus1, 0)

        # Calculate required capacity with safety factor
        # Use 3x the higher demand to ensure adequate capacity
        safety_factor = 3.0
        required_capacity = max(bus0_demand, bus1_demand) * safety_factor

        # Ensure minimum reasonable capacity (1000 MW)
        required_capacity = max(required_capacity, 1000)

        print(f"\n Fixing: {line_name}")
        print(f"    Connected buses: {bus0} ↔ {bus1}")
        print(f"    Bus demands: {bus0}: {bus0_demand:.1f} MW, {bus1}: {bus1_demand:.1f} MW")

        # Set s_nom to required capacity
        old_s_nom = network.lines.loc[line_name, 's_nom']
        network.lines.loc[line_name, 's_nom'] = required_capacity
        print(f"    s_nom: {old_s_nom} → {required_capacity:.1f} MW")

        # Make sure line is not extendable
        if 's_nom_extendable' not in network.lines.columns:
            network.lines['s_nom_extendable'] = False
        network.lines.loc[line_name, 's_nom_extendable'] = False
        print(f"    s_nom_extendable: → False")

    return network

def remove_offshore_wind(network):
    """
    Remove offshore wind generators. 
    All of these have zero nominal capacity (likely missing data). 
    Need to remove them to avoid division by zero error in constraint check for slack gens.
    Problem is still feasible without offwind slack since pypsa optimize still feasible.
    """
    
    # First, identify offshore wind generators
    offwind_gens = network.generators[
        network.generators.index.str.contains('offwind', case=False, na=False)
    ].index
    
    print(f"Found {len(offwind_gens)} offshore wind generators:")
    print(offwind_gens.tolist())
    
    # Check their properties
    offwind_data = network.generators.loc[offwind_gens, ['p_nom', 'control', 'carrier']]
    print("\nOffshore wind generator details:")
    print(offwind_data)
    
    # Remove offshore wind generators one by one
    print(f"\nRemoving {len(offwind_gens)} offshore wind generators...")
    for gen in offwind_gens:
        network.remove("Generator", gen)

def create_pypsa_network(network_file):
    """Create a PyPSA network from the .nc file."""
    # Initialize network
    network = pypsa.Network(network_file)
    for storage_name in network.storage_units.index:
        # Use .loc for direct assignment to avoid SettingWithCopyWarning
        network.storage_units.loc[storage_name, 'cyclic_state_of_charge'] = False

        # Set marginal_cost to 0.01
        network.storage_units.loc[storage_name, 'marginal_cost'] = 0.01

        # Set marginal_cost_storage to 0.01
        network.storage_units.loc[storage_name, 'marginal_cost_storage'] = 0.01

        # Set spill_cost to 0.1
        network.storage_units.loc[storage_name, 'spill_cost'] = 0.1

        network.storage_units.loc[storage_name, 'efficiency_store'] = 0.866025 #use phs efficiency (hydro didnt have an efficiency, but i want to model them all as the same)

        # Fix unrealistic max_hours values
        current_max_hours = network.storage_units.loc[storage_name, 'max_hours']

        if 'PHS' in storage_name:
            # PHS with missing data - set to typical range
            network.storage_units.loc[storage_name, 'max_hours'] = 8.0
            print(f"Fixed {storage_name}: set max_hours to 8.0")

        elif 'hydro' in storage_name:
            # Hydro with unrealistic data - set to validated range
            network.storage_units.loc[storage_name, 'max_hours'] = 6.0
            print(f"Fixed {storage_name}: corrected max_hours from {current_max_hours} to 6.0")


    fix_artificial_lines_reasonable(network)
    remove_offshore_wind(network)

    return network

In [3]:
network = create_pypsa_network("elec_s_10_ec_lc1.0_1h.nc")

Fixed ZA0 0 PHS: set max_hours to 8.0
Fixed ZA0 5 PHS: set max_hours to 8.0
Fixed ZA0 6 hydro: corrected max_hours from 3831.6270020496813 to 6.0
=== FIXING ARTIFICIAL LINES WITH REASONABLE CAPACITY ===
Found 3 artificial lines to fix:

 Fixing: lines new ZA0 4 <-> ZA2 0 AC
    Connected buses: ZA0 4 ↔ ZA2 0
    Bus demands: ZA0 4: 15945.8 MW, ZA2 0: 452.6 MW
    s_nom: 0.0 → 47837.3 MW
    s_nom_extendable: → False

 Fixing: lines new ZA0 0 <-> ZA1 0 AC
    Connected buses: ZA0 0 ↔ ZA1 0
    Bus demands: ZA0 0: 3513.0 MW, ZA1 0: 1386.9 MW
    s_nom: 0.0 → 10538.9 MW
    s_nom_extendable: → False

 Fixing: lines new ZA0 0 <-> ZA3 0 AC
    Connected buses: ZA0 0 ↔ ZA3 0
    Bus demands: ZA0 0: 3513.0 MW, ZA3 0: 721.1 MW
    s_nom: 0.0 → 10538.9 MW
    s_nom_extendable: → False
Found 12 offshore wind generators:
['ZA0 1 offwind-ac', 'ZA0 1 offwind-dc', 'ZA0 5 offwind-ac', 'ZA0 5 offwind-dc', 'ZA0 7 offwind-ac', 'ZA0 7 offwind-dc', 'ZA0 8 offwind-ac', 'ZA0 8 offwind-dc', 'ZA1 0 offwind-ac

In [12]:
def check_line_constraints(network, current_snapshot):
    constraint_results = np.zeros(len(network.lines.index))
    for idx, line_name in enumerate(network.lines.index):
            # Get line parameters
            s_nom = network.lines.loc[line_name, 's_nom']
            
            # Get s_max_pu if it exists in time series, otherwise use 1.0
            if hasattr(network.lines_t, 's_max_pu') and line_name in network.lines_t.s_max_pu.columns:
                s_max_pu = network.lines_t.s_max_pu.loc[current_snapshot, line_name]
            else:
                s_max_pu = network.lines.loc[line_name, 's_max_pu'] if 's_max_pu' in network.lines.columns else 1.0

            # Calculate the actual flow limit
            s_max = s_max_pu * s_nom

            # Get power flows
            p0 = network.lines_t.p0.loc[current_snapshot, line_name]
            
            # In PyPSA's linear power flow, the constraint is typically |P| ≤ S_nom
            # since reactive power is not modeled in LPF
            flow_magnitude = abs(p0)

            # Check if flow exceeds limit
            if flow_magnitude > s_max:
                violation = min(float((flow_magnitude - s_max)/s_max), 1.0)
                constraint_results[idx] = violation  # Use idx instead of constraint_results_idx

    return constraint_results

In [16]:
network.determine_network_topology()
        
# Step 2: Pre-compute power flow matrices for each sub-network
for sub in network.sub_networks.obj:
    sub.calculate_B_H()

# Initialize p_set for generators if it doesn't exist
if 'p_set' not in network.generators_t:
    print("Creating generators_t.p_set since it doesn't exist")
    network.generators_t['p_set'] = pd.DataFrame(index=network.snapshots, 
                                               columns=network.generators.index,
                                               data=0.0)
            
renewable_gens = network.generators_t.p_max_pu.columns

slack_generators = network.generators[network.generators.control == "Slack"].index
# in the 10-node SA network there are 4 slack gens so this should return a list of indexes
dispatchable_gens = network.generators[
    (~network.generators.index.isin(slack_generators)) &
    (~network.generators.index.isin(renewable_gens))
].index

# Renewable generators: have time-varying p_max_pu, not slack
renewable_gens = network.generators[
    (network.generators.index.isin(renewable_gens)) &
    (~network.generators.index.isin(slack_generators))
].index

# Storage units (if any exist in the network)
storage_units = network.storage_units.index

# Store names as lists for easier indexing
dispatchable_names = list(dispatchable_gens)
renewable_names = list(renewable_gens)
storage_names = list(storage_units)

# Get static limits for dispatchable generators
dispatchable_df = network.generators.loc[dispatchable_gens]
disp_p_min = (dispatchable_df.p_min_pu * dispatchable_df.p_nom).values #returns numpy arrays
disp_p_max = (dispatchable_df.p_max_pu * dispatchable_df.p_nom).values

# Get nominal capacities and minimum limits for renewable generators
renewable_df = network.generators.loc[renewable_gens]
renewable_p_nom = renewable_df.p_nom.values
renewable_p_min_pu = renewable_df.p_min_pu.values

storage_df = network.storage_units.loc[storage_units]
#this is a bit redundant since self.storage_units is the array of all indices of self.network.storage_units but leave it in so could replace which indices you want
storage_p_nom = storage_df.p_nom.values

def take_worst_action(network, current_snapshot, dispatchable_names, renewable_names, storage_names, disp_p_min, disp_p_max, renewable_p_nom, renewable_p_min_pu, storage_p_nom):
    """Set all generators to minimum power to create worst-case scenario"""
    # Set all dispatchable generators to minimum power
    for i, gen_name in enumerate(dispatchable_names):
        min_power = disp_p_min[i]
        network.generators_t.p_set.loc[current_snapshot, gen_name] = min_power
    
    # Set all renewable generators to minimum (usually zero)
    for i, gen_name in enumerate(renewable_names):
        min_power = renewable_p_min_pu[i] * renewable_p_nom[i]
        network.generators_t.p_set.loc[current_snapshot, gen_name] = min_power
    
    # Set all storage units to zero dispatch and zero set
    for storage_name in storage_names:
        # Initialize time-dependent storage parameters if they don't exist
        for attr in ['p_dispatch', 'p_set', 'p_store']:
            if attr not in network.storage_units_t:
                network.storage_units_t[attr] = pd.DataFrame(index=network.snapshots,
                                                           columns=network.storage_units.index,
                                                           data=0.0)
        
        network.storage_units_t.p_dispatch.loc[current_snapshot, storage_name] = 0.0
        network.storage_units_t.p_set.loc[current_snapshot, storage_name] = 0.0
        network.storage_units_t.p_store.loc[current_snapshot, storage_name] = 0.0
    
    # Run power flow
    try:
        network.lpf(current_snapshot, skip_pre=True)
        converged = True
    except Exception as e:
        print(f"Power flow failed for snapshot {current_snapshot}: {e}")
        converged = False
    
    return converged

In [19]:
# Reset network and initialize variables
violation_sum = 0

# Print information about the generators for debugging
print(f"Dispatchable generators: {dispatchable_names}")
print(f"Renewable generators: {renewable_names}")
print(f"Storage units: {storage_names}")

# # Check if we need to limit the number of snapshots to process
# max_snapshots = 24  # Process only one day (24 hours) to avoid excessive warnings
# snapshots_to_process = network.snapshots[:max_snapshots]
# print(f"Processing {len(snapshots_to_process)} snapshots out of {len(network.snapshots)} total")

# Create p_set if it doesn't exist
if 'p_set' not in network.generators_t:
    print("Creating generators_t.p_set since it doesn't exist")
    network.generators_t['p_set'] = pd.DataFrame(index=network.snapshots, 
                                               columns=network.generators.index,
                                               data=0.0)

# Loop through snapshots
for snapshot_idx, current_snapshot in enumerate(network.snapshots):
    # Take worst action for this snapshot
    converged = take_worst_action(network, current_snapshot, dispatchable_names, renewable_names, 
                     storage_names, disp_p_min, disp_p_max, renewable_p_nom, 
                     renewable_p_min_pu, storage_p_nom)
    
    if converged:
        # Check for constraint violations
        constraint_results = check_line_constraints(network, current_snapshot)
        violation_sum += np.sum(constraint_results)
    else:
        print(f"Skipping constraint check for snapshot {snapshot_idx} due to power flow failure")

print(f"Total line constraint violations: {violation_sum}")


Dispatchable generators: ['ZA0 1 CCGT', 'ZA0 3 biomass', 'ZA0 3 coal', 'ZA0 4 CCGT', 'ZA0 4 coal', 'ZA0 5 CCGT', 'ZA0 5 nuclear', 'ZA0 7 CCGT', 'ZA0 7 coal', 'ZA0 8 CCGT', 'ZA0 8 biomass', 'ZA0 9 coal']
Renewable generators: ['ZA0 0 onwind', 'ZA0 0 ror', 'ZA0 0 solar', 'ZA0 1 onwind', 'ZA0 1 solar', 'ZA0 2 onwind', 'ZA0 2 ror', 'ZA0 2 solar', 'ZA0 3 onwind', 'ZA0 3 ror', 'ZA0 3 solar', 'ZA0 4 onwind', 'ZA0 4 solar', 'ZA0 5 onwind', 'ZA0 5 solar', 'ZA0 6 onwind', 'ZA0 6 solar', 'ZA0 7 onwind', 'ZA0 7 solar', 'ZA0 8 onwind', 'ZA0 8 ror', 'ZA0 8 solar', 'ZA0 9 onwind', 'ZA0 9 solar', 'ZA1 0 onwind', 'ZA1 0 solar', 'ZA2 0 onwind', 'ZA2 0 solar', 'ZA3 0 onwind', 'ZA3 0 solar']
Storage units: ['ZA0 0 PHS', 'ZA0 5 PHS', 'ZA0 6 hydro']
Total line constraint violations: 330.872225422805
